In [26]:
import time
import numpy as np

from coppeliasim_zmqremoteapi_client import RemoteAPIClient 

def Rz(theta):
  
    return np.array([[ np.cos(theta), -np.sin(theta), 0 ],
                      [ np.sin(theta), np.cos(theta) , 0 ],
                      [ 0            , 0             , 1 ]])

In [27]:
try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)

    # Handle para o ROBÔ
    robotname = 'robotino'
    robotHandle = sim.getObject('/' + robotname)
    
    # Handle para as juntas das RODAS
    wheel1 = sim.getObject('/' + robotname + '/wheel0_joint')
    wheel2 = sim.getObject('/' + robotname + '/wheel1_joint')
    wheel3 = sim.getObject('/' + robotname + '/wheel2_joint')
    
    # Dados Robotino
    L = 0.135   # Metros
    r = 0.040   # Metros
               
    # Cinemática Direta
    Mdir = np.array([[-r/np.sqrt(3), 0, r/np.sqrt(3)], [r/3, (-2*r)/3, r/3], [r/(3*L), r/(3*L), r/(3*L)]])

    # Configuração inicial (x, y, w)
    q = np.array([0, 0, 0])

    # Goal configuration (x, y, theta)    
    qgoal = np.array([3, 3, np.deg2rad(90)])
    
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    
    # Inicia a simulação
    sim.startSimulation()
    sim.step()


    # Frame que representa o Goal
    goalFrame = sim.getObject('/Goal')
    sim.setObjectPosition(goalFrame, [qgoal[0], qgoal[1], 0], sim.handle_world)
    sim.setObjectOrientation(goalFrame, [0, 0, qgoal[2]], sim.handle_world)
    gain = np.array([[0.1, 0, 0], [0, 0.1, 0], [0, 0, 0.1]])
    
    print("Starting robot control loop...")
    while (sim_time := sim.getSimulationTime()) <= 90:
        print(f"Simulation time: {sim_time:.2f} [s]")
        
        dt = sim.getSimulationTimeStep()
                
        pos = sim.getObjectPosition(robotHandle, sim.handle_world)
        ori = sim.getObjectOrientation(robotHandle, sim.handle_world)

        q = np.array([pos[0], pos[1], ori[2]])
        
        error = qgoal - q
        print(f"Error: {np.linalg.norm(error[:2])}")
        
        # Margem aceitável de distância
        if (np.linalg.norm(error[:2]) < 0.05):
            break

        # Controller
        qdot = gain @ error
        
        # Cinemática Inversa
        # w1, w2, w3
        Minv = np.linalg.inv(Rz(q[2]) @ Mdir)
        u = Minv @ qdot
        
        # Enviando velocidades
        sim.setJointTargetVelocity(wheel1, u[0])
        sim.setJointTargetVelocity(wheel2, u[1])
        sim.setJointTargetVelocity(wheel3, u[2]) 
                
        sim.step()

    # Parando o robô
    print("Stopping robot...")
    sim.setJointTargetVelocity(wheel1, 0)
    sim.setJointTargetVelocity(wheel2, 0)
    sim.setJointTargetVelocity(wheel3, 0)
    
    print('CALC Pos: ', sim_time, q[:2], np.rad2deg(q[2]))
      
    pos = sim.getObjectPosition(robotHandle, sim.handle_world)
    print('SIM Pos: ', pos)

    ori = sim.getObjectOrientation(robotHandle, sim.handle_world)
    print('SIM Ori: ', np.rad2deg(ori))
    

except Exception as e:
    print(f"An error occurred: {e}")
    
# Parando a simulação
sim.stopSimulation()

print('Program ended')

Starting robot control loop...
Simulation time: 0.05 [s]
Error: 4.2426637441418436
Simulation time: 0.10 [s]
Error: 4.240970916134558
Simulation time: 0.15 [s]
Error: 4.236431648739739
Simulation time: 0.20 [s]
Error: 4.229258211615873
Simulation time: 0.25 [s]
Error: 4.219788216269801
Simulation time: 0.30 [s]
Error: 4.208275312733615
Simulation time: 0.35 [s]
Error: 4.194879751007918
Simulation time: 0.40 [s]
Error: 4.179646851052557
Simulation time: 0.45 [s]
Error: 4.162779182488967
Simulation time: 0.50 [s]
Error: 4.144873545211595
Simulation time: 0.55 [s]
Error: 4.1265375687113925
Simulation time: 0.60 [s]
Error: 4.107950508579681
Simulation time: 0.65 [s]
Error: 4.089098741476614
Simulation time: 0.70 [s]
Error: 4.070205968349627
Simulation time: 0.75 [s]
Error: 4.051231490704332
Simulation time: 0.80 [s]
Error: 4.031938742449197
Simulation time: 0.85 [s]
Error: 4.0124075163237185
Simulation time: 0.90 [s]
Error: 3.992733639569452
Simulation time: 0.95 [s]
Error: 3.9730700782251